This notebook shows how to train and convert a TensorFlow model to be used in an Arduino sketch with the EloquentTensorFlow32 or EloquentTensorFlowCortexM libraries.

In [1]:
! pip install "everywhereml>=0.2.32"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.2 MB/s eta 0:00:00
  Created wheel for everywhereml: filename=everywhereml-0.2.39-py3-none-any.whl size=81224 sha256=dd94dc0983a3e6b12606765873c222bf629653dadb999dc38a30349d6337beaa
  Stored in directory: /root/.cache/pip/wheels/96/16/0d/c55612f3c08c6ed348c45c7de6018dac22ea839124bc507ced
  Created wheel for hexdump: filename=hexdump-3.3-py3-none-any.whl size=8896 sha256=206c53330f85edc96bd847a3b46e227e915360daceff176ed9350b164db9a020
  Stored in directory: /root/.cache/pip/wheels/26/28/f7/f47d7ecd9ae44c4457e72c8bb617ef18ab332ee2b2a1047e87
Successfully built everywhereml hexdump


In [2]:
"""
Get X and y data
(replace with your own)
"""
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler


def load_Xy():
  """
  Load your data here.
  It may be a CSV file or whatever
  """
  return load_iris(return_X_y=True)


def get_Xy():
  """
  Normalize X
  One-hot encode y
  """
  X, y = load_Xy()
  X = np.asarray(X)
  y = np.asarray(y)
  num_classes = y.max()
  eye = np.eye(num_classes + 1)
  X_norm = MinMaxScaler().fit_transform(X)
  y_hot = np.asarray([eye[yi] for yi in y], dtype=int)

  return X_norm, y_hot

In [3]:
"""
Instantiate NN for classification
Replace with your own topology
"""
import tensorflow as tf
from tensorflow.keras import layers


def instantiate_nn_for_classification(input_shape, num_classes):
  model = tf.keras.Sequential()
  model.add(layers.Dense(32, activation='relu', input_shape=input_shape))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(num_classes, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [4]:
"""
Split data between train, validation and test
"""
from sklearn.model_selection import train_test_split


X, y = get_Xy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [5]:
"""
Train model
"""
input_shape = X.shape[1:]
num_classes = y.shape[1]

model = instantiate_nn_for_classification(input_shape, num_classes)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/50
5/5 [==============================] - 10s 734ms/step - loss: 1.0473 - accuracy: 0.6712 - val_loss: 1.1220 - val_accuracy: 0.4375
Epoch 2/50
5/5 [==============================] - 0s 83ms/step - loss: 1.0228 - accuracy: 0.6575 - val_loss: 1.1099 - val_accuracy: 0.4375
Epoch 3/50
5/5 [==============================] - 0s 73ms/step - loss: 1.0006 - accuracy: 0.6575 - val_loss: 1.0966 - val_accuracy: 0.4375
Epoch 4/50
5/5 [==============================] - 0s 87ms/step - loss: 0.9786 - accuracy: 0.6849 - val_loss: 1.0794 - val_accuracy: 0.4688
Epoch 5/50
5/5 [==============================] - 1s 138ms/step - loss: 0.9574 - accuracy: 0.6849 - val_loss: 1.0631 - val_accuracy: 0.5000
Epoch 6/50
5/5 [==============================] - 1s 140ms/step - loss: 0.9364 - accuracy: 0.6849 - val_loss: 1.0454 - val_accuracy: 0.5000
Epoch 7/50
5/5 [==============================] - 0s 67ms/step - loss: 0.9151 - accuracy: 0.6986 - val_loss: 1.0271 - val_accuracy: 0.5312
Epoch 8/50
5/5 [=======

In [6]:
"""
Export NN to C++
Copy-paste the generated code inside a file named model.h or irisModel.h
in your Arduino project
"""
from everywhereml.code_generators.tensorflow import convert_model


c_header = convert_model(model, X, y, model_name='irisModel')
print(c_header)

#pragma once

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

// automatically configure network
#define TF_NUM_INPUTS 4
#define TF_NUM_OUTPUTS 3
#define TF_NUM_OPS 2
#define TF_OP_SOFTMAX
#define TF_OP_FULLYCONNECTED




// sample data
float x0[4] = {0.22222222222f, 0.62500000000f, 0.06779661017f, 0.04166666667f};
float x1[4] = {0.75000000000f, 0.50000000000f, 0.62711864407f, 0.54166666667f};
float x2[4] = {0.55555555556f, 0.54166666667f, 0.84745762712f, 1.00000000000f};


/** model size = 5048 bytes **/
const unsigned char irisModel[] DATA_ALIGN_ATTRIBUTE = { 0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c,